### Bag of Words

Para realizar operações entre palavras, precisamos tratar os dados e, por tratar, entenda-se que estamos identificando as palavras por seus tipos (pronomes, substantivos, verbos, preposições, advérbios e adjetivos)

#### Pré-processamento

- Obter as string correspondentes ao texto de cada documento
- Extrair as palavras de cada string
- Remover pontuação e símbolos
- Remover "Stop Words"
- Extrair bigramas, trigramas, etc
- Normalização Léxica

In [11]:
import glob

files = glob.glob('DocCol1/*')
print(files[:5])

docs = ' '.join([f"""{open(fname,'r').read()}""" for fname in files])

print(docs)

, Alexandria, VA 22304-6145. And it's
	    also available from the NTIS, with yet another number.

	"Advanced Space Propulsion Study, Antiproton and Beamed Power
	    Propulsion", Robert Forward

	    AFAL TR-87-070 from the Air Force Astronautics Laboratory, DTIC
	    #AD-A189 218.
	    NTIS AD-A189 218/1	  PC A10/MF A01

	    Summarizes the previous paper, goes into detail on beamed power
	    systems including " 1) pellet, microwave, and laser beamed power
	    systems for intersteller transport; 2) a design for a
	    near-relativistic laser-pushed lightsail using near-term laser
	    technology; 3) a survey of laser thermal propulsion, tether
	    transportation systems, antiproton annihilation propulsion,
	    exotic applications of solar sails, and laser-pushed
	    interstellar lightsails; 4) the status of antiproton
	    annihilation propulsion as of 1986; and 5) the prospects for
	    obtaining antimatter ions heavier than antiprotons." Again,
	    there is an extensive bibli

In [12]:
import nltk

words = nltk.word_tokenize(docs)
print(words[:20])

['Subject', ':', 'Re', ':', 'Eck', 'vs', 'Rickey', '(', 'was', 'Re', ':', 'Rickey', "'s", 'whining', 'again', ')', 'From', ':', 'smith', '@']


In [13]:
import string

# o método isalpha verifica se a string em questão possui apenas caracteres alfabéticos [a-zA-Z], enquanto string.ascii_letters representa caracteres 'isolados' 
# que estejam presentes na sentença (ex: 'a', 'e'), o que não nos agrega sentido semântico
words = [w.lower() for w in words if w.isalpha() and w not in string.ascii_letters]

print(words[:20])

['subject', 're', 'eck', 'vs', 'rickey', 'was', 're', 'rickey', 'whining', 'again', 'from', 'smith', 'brian', 'smith', 'in', 'article', 'david', 'tate', 'writes', 'read']


### Removendo Stop-Words

Stop-words é um grupo de palavras que não representa sentido semântico dentro da linguagem, o que é incoveniente em termos de análise de dados

In [14]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

words = [w for w in words if w not in stop_words]
print(words[:20])

['subject', 'eck', 'vs', 'rickey', 'rickey', 'whining', 'smith', 'brian', 'smith', 'article', 'david', 'tate', 'writes', 'read', 'followups', 'thought', 'go', 'back', 'original', 'article']


### Bigramas, Trigramas ...

Expressões compostas por mais de uma palavra, o que acaba sendo extremamente relevante em determinados aspectos

In [15]:
from nltk import word_tokenize
from nltk.collocations import *

bigrams_measures = nltk.collocations.BigramAssocMeasures()


finder = BigramCollocationFinder.from_words(words)

for i in finder.nbest(bigrams_measures.raw_freq, 20):
    words.append(f"{i[0].lower()} {i[1].lower()}")


# mostra os 10 bigramas mais relevantes (que apareceram)
print(words[-10:])

['national space', 'degrees fahrenheit', 'space center', 'astronomical society', 'degrees celsius', 'dewey larson', 'mark singer', 'thu apr', 'celsius degrees', 'jon leech']


### Normalização Léxica

- Stemming: basicamente trata-se de converter as flexões verbais para a forma de infinitivo, para facilitar a análise semântica e verificar a relevância de um determinado verbo no contexto da análise (ex: ando, andando, andei, andarei -> andar)

- Lematização

In [16]:
from nltk.stem import PorterStemmer

words = [PorterStemmer().stem(w) for w in words]

print(words[:100])

['subject', 'eck', 'vs', 'rickey', 'rickey', 'whine', 'smith', 'brian', 'smith', 'articl', 'david', 'tate', 'write', 'read', 'followup', 'thought', 'go', 'back', 'origin', 'articl', 'make', 'specif', 'comment', 'method', 'jao', 'john', 'oswalt', 'said', 'obtain', 'play', 'play', 'record', 'comput', 'readabl', 'form', 'everi', 'major', 'leagu', 'basebal', 'game', 'past', 'sever', 'year', 'devis', 'algorithm', 'call', 'wrote', 'comput', 'program', 'calcul', 'everi', 'major', 'leagu', 'player', 'contribut', 'use', 'work', 'like', 'look', 'everi', 'situat', 'everi', 'game', 'basebal', 'season', 'situat', 'determin', 'inning', 'score', 'baserunn', 'mani', 'out', 'situat', 'count', 'mani', 'time', 'team', 'eventu', 'game', 'situat', 'occur', 'divid', 'number', 'time', 'situat', 'came', 'come', 'valu', 'situat', 'first', 'done', 'georg', 'lindsey', 'late', 'report', 'articl', 'investig', 'strategi', 'basebal', 'author']


Construindo a Matriz Documentos x Palavras

- A construção de uma "Bag of Words" é feita aplicando todos os passos acima em cada documento, juntando todas as informações em um DataFrame (ou array), ou seja:

### TD E IDF

- TF: frequência com que uma palavra aparece em um determinado documento 
    - count(palavra)/len(documento)

- IDF: mede o quão informativa cada palavra é no conjunto de documentos
    - Número de documentos / Número de vezes que uma palavra aparece em todos os documentos (1 ~ Número de documentos)
    
- TFIDF = TF * log(IDF)